# Part 2 - (SIMPLIFIED) BART Graph, Populations, Harmonic Centrality, & Louvain Modularity

#### Simplified BART Graph
This notebook is being used to first create a simplified version of the BART stations graph where each station is represented by a single node.  This graph also has single links between nodes to represent any potential line that connects them.  The weight for each link is the transfer time. 
**DEVELOP: We can also include a 'penalty' factor to the travel times to account for the fact that some links represent 1 train line, while others represent 4.**

#### BART Station Populations
Each BART station node in the graph is also provided a population attribute.  This population was quantified by finding all zip codes whose geograpic center lies within a box **1** miles from a given station and summing their populations.

#### Harmonic Centrality for Main Hub
We are applying the Harmonic Centrality Algorithm to this simplified BART graph to determine the center (based on travel time) to represent the best location for our main distribution hub. 

#### Shortest Path (Dijkstra's algorithm) for Directional Populations
After calculating the Harmonic Centrality , we double check populations in each direction from the indentified central hub, to potentially manually adjust our hub location to be more centered on population.  We are using the shortest path to loosely quantify populations from the central hub to the edges of the graph. 

#### Louvain Modularity for Communities
We are then applying the Louvain Modularity Algorithm to best cluster the BART Stations into communities and sub-communities.  This will help us in determining the best satelite locations to support the HUB.

#### Harmonic Centrality for Satellite Locations
We are running harmonic centrality on each of the communities created above to select the satelite locations for each. 

---
## Included Modules and Packages

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2
from geographiclib.geodesic import Geodesic

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

---
## Supporting Code

In [4]:
#     
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [5]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [6]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [7]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [8]:
def my_neo4j_create_node(station_name, population):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name, population: $population})
    
    """
    
    session.run(query, station_name=station_name, population=population)

In [9]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [10]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [12]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
#     print("\n-------------------------------------------------------------------------------")
#     print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
#     print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
#         print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
    return int(total_population)
    
#     print("\n-------------------------------------------------------------------------------")
#     print("  Total Population: ", f'{total_population:10,}')
#     print("-------------------------------------------------------------------------------")

In [13]:
def finding_directional_populations(source):
    '''
    This function sums the populations in the paths from an identified source to the various "edges" of the BART routes.
    It uses the shortest path algorithm and returns a df of the results. 
    '''
    
    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        REDUCE(sum = 0, pop in [nodeId IN nodeIds | gds.util.asNode(nodeId).population] | sum + pop) AS total_population,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).population] AS populations  

    ORDER BY index

    """
    #identifying the furthest stretches of BART
    #we will be ignoring the two airports in this analysis
    edges = {'NW': 'Richmond', 'NE': 'Antioch', 'SW': 'Millbrae', 'SE': 'Berryessa', 'E': 'Dublin'}

    #looping through the edges, running shortest path for each, and calculating populations
    results = pd.DataFrame(columns = ['from', 'to', 'total_population', 'nodes', 'populations'])
    for direction, station in edges.items():
        source = source
        target = station
        temp_df = my_neo4j_run_query_pandas(query, source=source, target=target)
        results = pd.concat([results, temp_df], axis=0)

    return results

In [14]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [15]:
cursor = connection.cursor()

---
## 1) BART Graph & Populations

### Clear Neo4j Database

In [16]:
my_neo4j_wipe_out_database()

### Creating Station Nodes w/ Population Attribute

In [17]:
connection.rollback()
#returns a df table of all stations
query = f"""

select station
from stations
order by station

"""
cursor.execute(query)
connection.rollback()
rows = cursor.fetchall()

#looping through each station
for row in rows:
    station = row[0]
    #calculating population within 1 mile of station
    population = my_station_get_zips(station, 1)
    #creating graph node for station with population attribute
    my_neo4j_create_node(station, population)

### Creating Line & Travel Time Links

In [18]:
connection.rollback()

#returns a dataframe of all connections and travel times between stations
query = """

select station_1 as from_station,
    station_2 as to_station,
    travel_time
from travel_times

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

#loops through each row in df returned and assigns from and to stations and travel time.
for row in rows:
    from_station = row[0]
    to_station = row[1]
    travel_time = int(row[2])
    
    #creates a new relationship in the graph for each
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)

### Checking Graph Created

In [19]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 102
-------------------------


### Neo4j interface at https://xxxx:7473

**Username: neo4j**

**Password: ucb_mids_w205**

**query to return all nodes and all relationships:**

```
match (n) return n
```

---
## 2) Harmonic Centrality

Using Harmonic Centrality to find the stations that are most at center of the BART system from a travel time perspective. 

In [20]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [21]:
query = """

CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).name AS name, 
    gds.util.asNode(nodeId).population AS population, 
    centrality as closeness
ORDER BY centrality DESC
limit 30

"""

harmonic_cent = my_neo4j_run_query_pandas(query)
harmonic_cent

,name,population,closeness
0,12th Street,16062,0.233302
1,Lake Merritt,16062,0.231867
2,West Oakland,26254,0.228991
3,MacArthur,22811,0.226052
4,19th Street,16062,0.219525
5,Coliseum,0,0.216537
6,Fruitvale,52299,0.214852
7,Bay Fair,41059,0.209259
8,Embarcadero,54398,0.204956
9,San Leandro,48088,0.204085


---
## 3) Shortest Path (Dijkstra's algorithm) 

Using this shortest path algorithm to quantify the populations that are in each direction from **12th Street**, which was identified as the central stations above.  This will help us determine if we should adjust our central distribution center to account for the distribution of populations. 

Note: This is just being used to roughly review populations.  There are a couple issues to acknowledge with the populations being calculated:

    1) Populations are tied to zip codes within proximity of stations.  It is possible that multiple stations are picking up the same zip codes and hence a duplication in populations.

    2) Certain branches from a central location share similiar stations (i.e. Richmond and Antioch both pass through MacArthur) so those populations are also being duplicated in this calculation. 

In [22]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [23]:
#checking for 12th street
finding_directional_populations('12th Street')

,from,to,total_population,nodes,populations
0,12th Street,Richmond,277893,"[12th Street, 19th Street, MacArthur, Ashby, D...","[16062, 16062, 22811, 68219, 82930, 50769, 210..."
0,12th Street,Antioch,323884,"[12th Street, 19th Street, MacArthur, Rockridg...","[16062, 16062, 22811, 39852, 19341, 29639, 227..."
0,12th Street,Millbrae,922589,"[12th Street, West Oakland, Embarcadero, Montg...","[16062, 26254, 54398, 85465, 140730, 74898, 63..."
0,12th Street,Berryessa,496043,"[12th Street, Lake Merritt, Fruitvale, Coliseu...","[16062, 16062, 52299, 0, 48088, 41059, 66056, ..."
0,12th Street,Dublin,227990,"[12th Street, Lake Merritt, Fruitvale, Coliseu...","[16062, 16062, 52299, 0, 48088, 41059, 0, 0, 5..."


In [24]:
#checking for West Oakland
finding_directional_populations('West Oakland')

,from,to,total_population,nodes,populations
0,West Oakland,Richmond,304147,"[West Oakland, 12th Street, 19th Street, MacAr...","[26254, 16062, 16062, 22811, 68219, 82930, 507..."
0,West Oakland,Antioch,350138,"[West Oakland, 12th Street, 19th Street, MacAr...","[26254, 16062, 16062, 22811, 39852, 19341, 296..."
0,West Oakland,Millbrae,906527,"[West Oakland, Embarcadero, Montgomery Street,...","[26254, 54398, 85465, 140730, 74898, 63489, 10..."
0,West Oakland,Berryessa,506235,"[West Oakland, Lake Merritt, Fruitvale, Colise...","[26254, 16062, 52299, 0, 48088, 41059, 66056, ..."
0,West Oakland,Dublin,238182,"[West Oakland, Lake Merritt, Fruitvale, Colise...","[26254, 16062, 52299, 0, 48088, 41059, 0, 0, 5..."


In [25]:
#checking for Embarcadero
finding_directional_populations('Embarcadero')

,from,to,total_population,nodes,populations
0,Embarcadero,Richmond,358545,"[Embarcadero, West Oakland, 12th Street, 19th ...","[54398, 26254, 16062, 16062, 22811, 68219, 829..."
0,Embarcadero,Antioch,404536,"[Embarcadero, West Oakland, 12th Street, 19th ...","[54398, 26254, 16062, 16062, 22811, 39852, 193..."
0,Embarcadero,Millbrae,880273,"[Embarcadero, Montgomery Street, Powell Street...","[54398, 85465, 140730, 74898, 63489, 108915, 1..."
0,Embarcadero,Berryessa,560633,"[Embarcadero, West Oakland, Lake Merritt, Frui...","[54398, 26254, 16062, 52299, 0, 48088, 41059, ..."
0,Embarcadero,Dublin,292580,"[Embarcadero, West Oakland, Lake Merritt, Frui...","[54398, 26254, 16062, 52299, 0, 48088, 41059, ..."


#### We are choosing to use West Oakland are our Main Distribution Hub. 

---
## 4) Louvain Modularity

Using to best cluster the BART Stations into communities and sub-communities.  This will help us in determining the best satelite locations to support the HUB.

In [26]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK',
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [27]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name,
    gds.util.asNode(nodeId).population AS population,
    communityId as community, 
    intermediateCommunityIds[0] as intermediate_community
ORDER BY community, name ASC

"""

louv_mod = my_neo4j_run_query_pandas(query)
louv_mod

,name,population,community,intermediate_community
0,Ashby,68219,2,15
1,Downtown Berkeley,82930,2,15
2,El Cerrito Plaza,21040,2,38
3,El Cerrito del Norte,0,2,38
4,North Berkeley,50769,2,15
5,Richmond,0,2,38
6,16th Street Mission,63489,10,3
7,24th Street Mission,108915,10,3
8,Civic Center,74898,10,10
9,Glen Park,115068,10,3


---
## 5) Harmonic Centrality for Satelite Locations
We are running harmonic centrality on each of the communities created above to select the satelite locations for each.

In [28]:
#quantifying all communities
communities = louv_mod['community'].unique()
communities

array([ 2, 10, 14, 23, 29, 31, 45, 46])

In [29]:
print("Chosen Satelite Location for each Community\n\n")
#looping through each community
for community in communities:
    print('='*70)
    print('Community #: ', community)
    #########STEP 1: Collect stations for community graph##########################
    ## identify nodes to the community
    stations = louv_mod.name[louv_mod.community == community].to_list()
    print('Stations: ', stations)

    #########STEP 2: Creating Station Nodes########################################
    #wipe neo4j database
    my_neo4j_wipe_out_database()

    connection.rollback()
    #returns a df table of all stations
    query = f"""

    select station
    from stations
    WHERE station IN ({', '.join(['%s'] * len(stations))})
    order by station

    """
    cursor.execute(query, stations)
    connection.rollback()
    rows = cursor.fetchall()

    #looping through each station
    for row in rows:
        station = row[0]
        #calculating population within 1 mile of station
        population = my_station_get_zips(station, 1)
        #creating graph node for station with population attribute
        my_neo4j_create_node(station, population)
        
    #########STEP 3: Creating Line and Travel Links###############################
    connection.rollback()
    #returns a dataframe of all connections and travel times between stations
    query = f"""

    select station_1 as from_station,
        station_2 as to_station,
        travel_time
    from travel_times
    where station_1 IN ({', '.join(['%s'] * len(stations))})
    and station_2 in ({', '.join(['%s'] * len(stations))})


    """
    cursor.execute(query, stations + stations)
    connection.rollback()
    rows = cursor.fetchall()

    #loops through each row in df returned and assigns from and to stations and travel time.
    for row in rows:
        from_station = row[0]
        to_station = row[1]
        travel_time = int(row[2])
        #creates a new relationship in the graph for each
        my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
        
    #########STEP 4: Harmonic Centrality#########################################

    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """
    CALL gds.alpha.closeness.harmonic.stream('ds_graph', {})
    YIELD nodeId, centrality
    RETURN gds.util.asNode(nodeId).name AS name, 
        gds.util.asNode(nodeId).population AS population, 
        centrality as closeness
    ORDER BY centrality DESC
    limit 1
    """

    satelite_cent = my_neo4j_run_query_pandas(query)

    #########STEP 5: APPENDING Satelite Location to df###################################################################
    print('Satelite Location: ', satelite_cent.iloc[0,0])
    print('-'*70)
    print('')
  

Chosen Satelite Location for each Community


Community #:  2
Stations:  ['Ashby', 'Downtown Berkeley', 'El Cerrito Plaza', 'El Cerrito del Norte', 'North Berkeley', 'Richmond']
Satelite Location:  El Cerrito Plaza
----------------------------------------------------------------------

Community #:  10
Stations:  ['16th Street Mission', '24th Street Mission', 'Civic Center', 'Glen Park', 'Powell Street']
Satelite Location:  16th Street Mission
----------------------------------------------------------------------

Community #:  14
Stations:  ['Balboa Park', 'Colma', 'Daly City', 'Millbrae', 'SFO', 'San Bruno', 'South San Francisco']
Satelite Location:  San Bruno
----------------------------------------------------------------------

Community #:  23
Stations:  ['Bay Fair', 'Castro Valley', 'Coliseum', 'Dublin', 'Fruitvale', 'OAK', 'San Leandro', 'West Dublin']
Satelite Location:  Coliseum
----------------------------------------------------------------------

Community #:  29
Stations:

In [30]:
#end of notebook